In [ ]:
%%capture
%pip install pandas matplotlib
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle
import pandas as pd
from matplotlib import pyplot as plt
from analysis_utils import (
    pivot_by_cond,
    load_logs_from_jsonl,
    load_results_from_pickle,
    calc_durations,
    plot_timeline,
)

In [ ]:
# load results and parameters from pickle
results = ["num_total_events", "user", "sys"]

df, params, dataset, sim_names = load_results_from_pickle()
print(df.dtypes)

In [ ]:
df[params]

In [ ]:
# you can specify the conditions to filter the dataframe
condition = {"purification_type": 1001, "connection_type": '"MIM"'}

if "num_node" in params:
    pivot_by_cond(df, params=params, x="num_node", value="user", cond=condition).plot(
        figsize=(15, 15), ylabel="duration (s)"
    )
else:
    print(f"no parameter found: num_node")

In [ ]:
logs = load_logs_from_jsonl(sim_names)

# specify a simulation to show timeline
sim_name = sim_names[0]
log = logs[sim_name]

durations = calc_durations(log)

fig, ax = plt.subplots(figsize=(35, 22))
plt.xlim([0, 0.001])
plot_timeline(ax, durations, "qubit_busy")
plot_timeline(ax, durations, "qubit_allocation")
plot_timeline(ax, durations, "bp_lifetime")
for e in durations["bp_annotations"]:
    text, x, y = e
    ax.annotate(text, xy=(x, y))

ax.legend(loc=(0.01, 0.01))
print(sim_name)